## Imports

In [0]:
from tensorflow.python.keras.layers import Dense, Flatten, MaxPooling2D
from tensorflow.python.keras import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt
import matplotlib.image as img
%matplotlib inline

import numpy as np

import os

import pandas as pd

import random

from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

Using TensorFlow backend.


In [0]:
from sklearn.model_selection import train_test_split

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
from tensorflow.keras.applications.resnet import ResNet101

## Load Datasets

In [0]:
#Dataset
downloaded = drive.CreateFile({'id':"1J615QNajDV3Q89TnE8R-iIvj_5zhMo1J"})   # replace the id with id of file you want to access
downloaded.GetContentFile('train_images.npy')        # replace the file name with your file

In [0]:
#Dataset
downloaded = drive.CreateFile({'id':"1tCHOmcsg96D9xfwY1Lf2-xZPT1PEUePe"})   # replace the id with id of file you want to access
downloaded.GetContentFile('train_labels.csv')        # replace the file name with your file

In [0]:
#Dataset
downloaded = drive.CreateFile({'id':"1ZdU78f7JN-OUI3Gd0UCFMwbr9RnoHk6G"})   # replace the id with id of file you want to access
downloaded.GetContentFile('test_images.npy')        # replace the file name with your file

## Preprocessing

In [0]:
testX=np.load('test_images.npy')

In [0]:
trainX=np.load('train_images.npy')

In [0]:
trainY=pd.read_csv('train_labels.csv')

In [0]:
validationX=trainX[40000:, :]
trainX=trainX[:40000, :]

In [0]:
validationY=trainY.iloc[40000:, 1]
trainY=trainY.iloc[:40000, 1]

In [0]:
def preprocessY (lst):
  integer_encoded = label_encoder.fit_transform(np.array(lst))
  trainY=to_categorical(integer_encoded)
  return trainY

In [0]:
def preprocessX(data):
  size=data.shape[0]
  data_new=np.zeros((size, 32, 32,3 ))
  for i in range (size):
    data_new[i]=np.dstack((data[i][0], data[i][1], data[i][2]))
    i=i+1
  
  return data_new

In [0]:
trainX=preprocessX(trainX)
validationX=preprocessX(validationX)

In [0]:
trainY=preprocessY(trainY)
validationY=preprocessY(validationY)

## Base Model: PreTrained Model

In [0]:
base_model = ResNet101(include_top=False, input_shape=(32, 32, 3))

171450368/171446536 [==============================] - 2s 0us/step


In [0]:
#Freeze all Layers of ResNET
base_model.trainable = False

In [0]:
base_model.summary()

Model: "resnet101"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 16, 16, 64)   256         conv1_conv[0][0]                 
__________________________________________________________________________________________

## Model

In [0]:
layer1=Flatten()
layer2=Dense(1024, activation='relu')
layer3=Dense(10, activation='softmax')

In [0]:
model = Sequential([
  base_model, layer1, layer2, layer3])

In [0]:
model.compile(loss=CategoricalCrossentropy(),
              optimizer=Adam(),
              metrics=['accuracy'])

In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet101 (Model)            (None, 1, 1, 2048)        42658176  
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              2098176   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                10250     
Total params: 44,766,602
Trainable params: 2,108,426
Non-trainable params: 42,658,176
_________________________________________________________________


## Checkpoints

In [0]:
# Include the epoch in the file name (uses `str.format`)

checkpoint_path = '/content/drive/My Drive/ML Projects/CIFAR/Checkpoints/cp-{epoch:04d}.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=1)


## Image Generator

In [0]:
gen = ImageDataGenerator(rotation_range=30, width_shift_range=0.08, shear_range=0.3,
                               height_shift_range=0.08, zoom_range=0.08) 

In [0]:
batch_size=64

## Reload Weights
### Whenever Colab crashed, weights were reloaded before resuming training.

In [0]:
model.load_weights('/content/drive/My Drive/ML Projects/CIFAR/Checkpoints/cp-0024.ckpt')

## Training Model
### After reloading the previous 24 epochs, we resume training.
### Training interrupted as accuracy had become roughly constant


In [0]:
history=model.fit_generator(gen.flow(trainX, trainY, batch_size), 
                    steps_per_epoch=trainY.shape[0]//batch_size, 
                    epochs=40, 
                    validation_data=gen.flow(validationX, validationY, batch_size),
                    validation_steps=validationY.shape[0]//batch_size,
                    callbacks=[cp_callback]
                    )

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/40
625/625 [==============================] - ETA: 0s - loss: 0.8851 - accuracy: 0.6903
Epoch 00001: saving model to /content/drive/My Drive/ML Projects/CIFAR/Checkpoints/cp-0001.ckpt
625/625 [==============================] - 351s 562ms/step - loss: 0.8851 - accuracy: 0.6903 - val_loss: 1.3388 - val_accuracy: 0.5751
Epoch 2/40
625/625 [==============================] - ETA: 0s - loss: 0.8772 - accuracy: 0.6913
Epoch 00002: saving model to /content/drive/My Drive/ML Projects/CIFAR/Checkpoints/cp-0002.ckpt
625/625 [==============================] - 347s 556ms/step - loss: 0.8772 - accuracy: 0.6913 - val_loss: 1.3362 - val_accuracy: 0.5736
Epoch 3/40
625/625 [==============================] - ETA: 0s - loss: 0.8779 - accuracy: 0.6894
Epoch 00003: saving model to /content/drive/My Drive/ML Projects/CIFAR/Checkpoints/cp-0003.ckpt
625/625 [==============================] - 350s 560ms/step - loss: 0.8779 - ac

KeyboardInterrupt: ignored